## 为什么要进行特征抽取
### 比如决策树算法，只能接收数字进行训练，这时候就要对特征值为str类型的数据进行特征抽取

In [288]:
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import jieba

### 字典抽取DictVectorizer

In [63]:
fruits=[{"fruit":"苹果","price":5,"eva":"cheap"}, {"fruit":"橘子","price":5.9,"eva":"cheap"}, {"fruit":"波萝","price":9.9,"eva":"expensive"}, {"fruit":"黄瓜","price":2.5,"eva":"cheap"}, {"fruit":"火龙果","price":12,"eva":"expensive"}]
fruits = pd.DataFrame(fruits)

features = fruits.drop("eva", axis=1)
target = fruits["eva"]

# 对于feature values需要将其进行特征抽取
features

,fruit,price
0,苹果,5.0
1,橘子,5.9
2,波萝,9.9
3,黄瓜,2.5
4,火龙果,12.0


In [46]:
# 首先转为字典类型
features_pre = features.to_dict(orient="record")

In [48]:
# 特征抽取
vect = sklearn.feature_extraction.DictVectorizer()
vect.fit(features_pre)
print(vect.get_feature_names())
result = vect.transform(features_pre)
print(result)
print(result.toarray())

['fruit=橘子', 'fruit=波萝', 'fruit=火龙果', 'fruit=苹果', 'fruit=黄瓜', 'price']
  (0, 3)	1.0
  (0, 5)	5.0
  (1, 0)	1.0
  (1, 5)	5.9
  (2, 1)	1.0
  (2, 5)	9.9
  (3, 4)	1.0
  (3, 5)	2.5
  (4, 2)	1.0
  (4, 5)	12.0
[[ 0.   0.   0.   1.   0.   5. ]
 [ 1.   0.   0.   0.   0.   5.9]
 [ 0.   1.   0.   0.   0.   9.9]
 [ 0.   0.   0.   0.   1.   2.5]
 [ 0.   0.   1.   0.   0.  12. ]]


In [57]:
# 进行决策树训练
tree = DecisionTreeClassifier()
tree.fit(result, np.array(target))

DecisionTreeClassifier()

In [75]:
# 预测
tree.predict([[0,1,0,0,0,8]])

array(['expensive'], dtype=object)

### 文本特征抽取CountVectorizer
用于文章分类等

In [79]:
text1 = 'life is short, i need python'
text2 = 'i love python, it makes me happy'
vect = CountVectorizer()
words = vect.fit_transform([text1,text2])
print(vect.get_feature_names())
print(words)
print(words.toarray())

['happy', 'is', 'it', 'life', 'love', 'makes', 'me', 'need', 'python', 'short']
  (0, 3)	1
  (0, 1)	1
  (0, 9)	1
  (0, 7)	1
  (0, 8)	1
  (1, 8)	1
  (1, 4)	1
  (1, 2)	1
  (1, 5)	1
  (1, 6)	1
  (1, 0)	1
[[0 1 0 1 0 0 0 1 1 1]
 [1 0 1 0 1 1 1 0 1 0]]


#### 对于中文的特征抽取，因为CountVectorizer只能识别两个空字符串之前的字符，所以要先对中文字符串进行处理，这里用到jieba包

In [95]:
# 准备-filter函数
filter_ = ["a","b"]
list_ = ["a","b","c","d","e"]
list1_ = "abcde"
for i in filter(lambda x: x not in filter_, list_):
    print(i)
    
print("-"*20)

for i in filter(lambda x: x not in filter_, list1_):
    print(i)

c
d
e
--------------------
c
d
e


In [102]:
# 准备-jieba的使用
word = "李雅婷说她不想努力了，我觉得她傻傻的"
word_jieba = list(jieba.cut(word))
# 符号对特征数据训练时无效的，所以要去除标点符号
# 先创建一个列表，把需要过滤的符号都放进去
biaodian = ["。", "，"]
# 过滤
list_ = list(filter(lambda x: x not in biaodian, word_jieba))
print(list_)
# 转为CountVectorizer能识别的格式
words = " ".join(list_)
print(words)

['李雅婷', '说', '她', '不想', '努力', '了', '我', '觉得', '她', '傻傻的']
李雅婷 说 她 不想 努力 了 我 觉得 她 傻傻的


In [108]:
# countvectorizer的在中文中的应用
word1 = "生的旅途是充满艰险、蜿蜒曲折的。面对他，与他挑战需要我们内心深处的勇气。而勇气的赋予者--对个人成长影响最大的一个人。这个人给予了我们挑战自己、战胜强敌的决心，是我们人生中最重要的精神支柱。"
word2 = "对我影响最大的一个人是《隐形的翅膀》中的女主角，一个失去了双臂的青春女孩，一个与命运搏斗的女孩。"
word3 = "对我影响最大的一个人是《隐形的翅膀》中的女主角，一个失去了双臂的青春女孩，一个与命运搏斗的女孩。"
words = [word1,word2,word3]
words_new = []
biaodian=["。","，","《","》"]
for word in words:
    words_new.append(" ".join(list(filter(lambda x: x not in biaodian, jieba.cut(word)))))
words_new

['生 的 旅途 是 充满 艰险 、 蜿蜒 曲折 的 面对 他 与 他 挑战 需要 我们 内心深处 的 勇气 而 勇气 的 赋予 者 -- 对 个人成长 影响 最大 的 一个 人 这个 人 给予 了 我们 挑战 自己 、 战胜 强敌 的 决心 是 我们 人生 中 最 重要 的 精神支柱',
 '对 我 影响 最大 的 一个 人 是 隐形 的 翅膀 中 的 女主角 一个 失去 了 双臂 的 青春 女孩 一个 与 命运 搏斗 的 女孩',
 '对 我 影响 最大 的 一个 人 是 隐形 的 翅膀 中 的 女主角 一个 失去 了 双臂 的 青春 女孩 一个 与 命运 搏斗 的 女孩']

In [111]:
vect = CountVectorizer()
vect.fit(words_new)
print(vect.get_feature_names())

result = vect.transform(words_new)
print(result.toarray())

['一个', '个人成长', '人生', '充满', '内心深处', '决心', '勇气', '双臂', '命运', '失去', '女主角', '女孩', '强敌', '影响', '我们', '战胜', '挑战', '搏斗', '旅途', '曲折', '最大', '精神支柱', '给予', '翅膀', '自己', '艰险', '蜿蜒', '赋予', '这个', '重要', '隐形', '需要', '青春', '面对']
[[1 1 1 1 1 1 2 0 0 0 0 0 1 1 3 1 2 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1]
 [3 0 0 0 0 0 0 1 1 1 1 2 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0]
 [3 0 0 0 0 0 0 1 1 1 1 2 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0]]


## 中文特征抽取实战

In [251]:
# 导入文件，生成features和target
with open("data/Chinese_vect/paper.txt","r",encoding="utf-8") as f:
    paper=f.read()
    
# 引入re模块对paper进行分割
import re
features = paper_list = re.split(r"\n\n+",paper)

# target列表，0代表励志累，1代表科技类
target =[0 for _ in range(10)] + [1 for _ in range(10)]

In [252]:
# 对中文features进行处理，整理成countvectorizer能识别的形式
biaodian=["一一","pl12","pl15","、","-----","·","----","— —","…","，","。","《","》","……","！","“","“","（","）","：","；","？","”","*","!","?",",","\n","\u3000",",","\t","1","2","3","4","5","6","7","8","9","10"]
features=[]
for paper in paper_list:
    features.append(" ".join(list(filter(lambda x: x not in biaodian, jieba.cut(paper)))))

In [253]:
# 特征抽取
vect = CountVectorizer()
features = vect.fit_transform(features)
print(vect.get_feature_names()[0:20])
features.toarray()

['100', '110', '177', '1997', '20', '2020', '21', '35', '45', '5g', '66', '95', 'dsi', '一万多', '一下', '一两个', '一个', '一个个', '一举', '一些']


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [254]:
# 分割
x_train,x_test,y_train,y_test = train_test_split(features,target,train_size=0.95)

In [255]:
multinomial = MultinomialNB()
multinomial.fit(x_train,y_train)
multinomial.score(x_test,y_test)

1.0

In [264]:
# 随便抽取文章进行识别
paper1 = "风月如流，光阴迈着一成不变的脚步，不疾不缓地走着，走过一个个春秋冬夏，走过一个个月阴日晴。蓦然回首，远逝的日子仿佛晃动的万花筒，不经意间摇晃出一个个无法模仿的图案，那一次次的聚散与悲欢都成绝版。于是我知道，青春是一列单程客车，不能返回也没有驿站。晨钟暮鼓，催促我扬起远航的风帆，心却如一枚留恋故枝的叶子，踟蹰在昨日的风中，依依难从我做起舍。也知道不该贻误行程，却又无法摆脱情感的羁绊，无法将曾经有过的一切尘封在记忆的城堡中，重生日锁紧，每天走过日落月升，仿佛伫立于清澈的河流中，那清清的河水就是我拥有而且仅仅拥有一次的生命啊，我却有太多的时候是立在水中，任凭上游的水流过我之后成为下游的水，任凭未来流经我的现在成为过去，我却因为过分沉迷于昨天而失去了今天和明天，于是一无所有。常常想象，当春华和秋实都成为历史之后，我披满头雪花坐在昨天的记忆中，咀嚼脸上每一道沟壑所深藏的故事，该是一种怎样的心境!本该是懂得很多便相信得很少，却依然一次次为“狼来了”的戏言所惑;本该是走过冬天更钟情于春天，却依然留恋“雪孩子”的童话。走过的路程似乎是一个圆，长途跋涉后又回到起点。于是很多时候是在做相同的事，就是寻找目标。曾经在幽幽暗暗中苦思冥想，终于破译出原来最简单的草枯草荣就是一种玄奥，那是生命的直观教具：花只能红一次，草只能绿一年。逝者如斯，如烟的往事都已随季风飘零，何必苦苦地追寻与回忆。青春的列车载客无数，没有人能预料沿途的遭遇，也没有人能重回自己上车的起点。既然已经别无选择，就让我在时间的钢轨上碾碎曾经有过的失意与如意，然后推开门窗，沐着新鲜的空气与阳光，迎接崭新的行程。因为我已经知道，青春是一列单程客车，不能返回也没有驿站。"
paper2 = "那些青春，已悄然老去;那些梦想，你可曾记得?青春，只一个很美的词，但又不知道它没在哪里。或许是因为年轻，所以享有青春。或许是因为年轻，才会对所有怀揣幻想。有些时候，曾幻想现实是多么美好，真想一下子融入现实生活中去感受一切。校园里的青涩年华，拥有美好的青春时光。我是幸福的，幸福到感受不到现实的残酷。而今，忽然回头，一些斑驳的记忆已支离破碎，再也拼凑不回。在我们的生命中，青春也悄然老去，离我们越拉越远，模糊了我们的实现，忘记了前路的方向。青春，是一个残酷的词。但又不明白它为何残酷。或许是因为成熟，所以知道残酷。或许是因为成熟，才会慢慢遗忘了青春。偶尔回头看看，却发现自己已偏离了轨道，和现实越走越近了，以至于丢失了那份纯真。梦想，似乎很重、很重。压得我们快要喘不过气。有时候甚至想丢开它，独自去一个无人知晓的地方。不知不觉中，梦想也随着我们长大了，越长大就越重。我们肩膀上的压力越大，脚步就不会变慢。这个季节，很快就会离我们而去，只是我们还停留在原地，依然守护着那些遥不可及的梦想。曾几何时，梦想拉近了我们与现实的距离，把我们一下子打回了原形。我们渐渐地，和现实越走越近。而在残酷的现实中，梦被扭曲了，变味儿了。无意之中，它渐渐偏离了我们。我们可以选择做一个自私的人，哪怕是一无所有。然后有一天突然发现，自己也变得很现实。于是只好重新拾起那些残缺的梦想。那些陈腐的过往，如今已随风即逝。雨过之后，我们看见的不是阳光，不是彩虹，而已灰色的天空，没有一丝色彩。繁花过后，青春散尽。那些要不可以的梦想，随着一阵微风而远行，不留一丝足迹。而在我们心底，仍然残留着一些破碎的痕迹。常说：我们有资本，因为我们还年轻。可是青春经不起肆意挥霍，也许有一天梦想会随着时间的流逝而不见。有一天，我们终将会慢慢老去。生活中，我们不应该是感伤的。忧伤和快乐事不过是一种形式。岁月带走了青春，别人看似美好的青春，但没有多少实际意义。奋斗，为时不晚，因为心中有梦。纵然年华不再，但是心中的斗志永不磨灭。它就像是一个火种，只要心底的那一丝温度不灭，它就会燃烧，越烧越旺。让我们接受过去，接受自己，带上梦想，悄悄地开始一段自己的旅程……"
paper3 = "四十年砥砺奋进，七十年长歌未央，一百年精神永存，时至今日，在五四爱国旗帜的引领下，中国人进步图存、科学救国。中国梦以一丝微光到今日的无限光明，离不开一代又一代人的接力传承，作为当代青年，我们承载着中华民族复兴之重任，科学兴国，吾辈担当。科学兴国，是历史的启迪。那时我们沉浸于“天朝上国”的美梦而不自知，殊不知，工业革命后科技发展迅速的西方文明已逐渐将我们甩在身后。当西方列强用坚船利炮轰开中国紧锁已久的大门，这片繁华千年的大地上留下了无尽的血泪与伤痕。近百年屈辱史，凝聚了中华民族无力的抗争与挣扎，终将沉睡的民族唤醒。鲁迅说：“我们将火药制成了烟花炮竹，可西方却将其制成火炮。”科学的停滞与落后，是造成我国国力落后于西方的根本原因，五四运动高举科学救国的旗帜，将中国唤醒。科学兴国，是时代的感召。我们关注今日之中国，经济飞速发展，已跃居世界前列。林达曾言：“刺破水面的锐利冰封，它也许只露出一角。”我们不仅仅要看到经济增长的高速度，更要看到速度之下，能源驱动型产业面临着资源枯竭、环境污染;劳动力密集型产业面临着人口红利衰退，增长乏力;高科技产业核心科技受制于发达国家的窘境。无论经济之花多么繁荣美丽，若没有科技作为支撑，中国经济持续发展，中国之梦的实现又何以为继?我们必须重新审视这个时代，这是一个科技飞速发展，日新月异的时代，科学技术作为第一生产力，已在无数实践中被证明，迈入知识经济时代，国之兴盛当以科学兴国、科学创新为先。科学兴国，是吾辈之担当。挪威戏剧家易卜生说过：“社会如同一条船，每个人都要做好掌舵的准备。”而我们要做的，则是要做好接力传承中华科学兴国之使命。我们可以看到，先辈们已为我们迈出了坚实一步，从五四科学救国的大旗，到“八六三”计划，科教兴国战略，再到神舟系列飞船升天，杂交水稻亩产再度突破。我看到，从邓稼先的隐姓埋名，到屠呦呦永不放弃，再到南仁东的一生坚守，无数前辈的努力，造就了今日蓬勃发展之中国。而我辈当承其志，将中国梦以高铁般速度持续推进。这是一个充满希望的国度，这是一个生机勃勃的时代，领袖在新年贺词中说：“我们都在努力奔跑，我们都是追梦人。”愿当代青年同胞们用科学力量武装自己，接力传承中华复兴之梦!"
paper4 = "开学第一课，一听到这个名字，每一个小学生都会记忆犹新，而他也整整陪伴了我5年。今年的主题是五星红旗，我为你自豪！说到五星红旗，有的人就会想到战争时期战士们的英勇牺牲；有的人就会想到党中央的正确领导；还有的人会想到中国刚成立百废待兴的时刻；而我想到的是我们国家日新月异的科技发展。这70年里，我国的科学技术从落后到了前三，从一无所有到世界领先，今年上半年，在第四次工业革命中，我们实现了弯道超车。中国的科技正在飞速发展，新四大发明正在影响着世界。在开学第一课里，那些大哥哥大姐姐手中的救援机器人成功实现了3分钟的迷你救援。看到他们将拼好的小机器人通过ipad操控就可以操控自如。尽管路上有很多障碍物，但是这些机器人靠这搭桥车通过断桥，靠推作文https://Www.ZuoWEn8.Com/土车把石头推到一边，靠机器爪子把一大块的墙移开救出人，然后送进车里飞快的去医院。我们都知道，当灾难降临后的72小时是救援的最好时机也就是3天，由于是迷你实验的成功也可以预测出我们在真实情况下也可以在3天之内救出被困者。我们不光有先进的救援设备，我们还有探索太空的神器——玉兔，当那个小家伙出现在舞台上的时候，我的眼睛都看直了。我也很想亲自到现场去看看，亲自操作一下小机器人，亲自和小玉兔进行互动……而更值得我学习的就是这些伟大的科技的背后，那个伟大的中国梦。所以作为巩华追梦少年的我要从新学期开始就努力严格要求自己，写一手好字，练就一副好口才，真正把爱国之志变成报国之行。今天为振兴中华而勤奋学习，明天为创造祖国辉煌未来贡献自己的力量！"
papers=[paper1,paper2,paper3,paper4]
papers_target=[0,0,1,1]
#特征抽取
features=[]
for paper in papers:
    features.append(" ".join(list(filter(lambda x: x not in biaodian, jieba.cut(paper)))))
features = vect.transform(features)
#评分
print("预测:",multinomial.predict(features))
print("评分:",multinomial.score(features,papers_target))
print(classification_report(papers_target,multinomial.predict(features),labels=[0,1],target_names=["励志类文章","科技类文章"]))

预测: [0 0 1 1]
评分: 1.0
              precision    recall  f1-score   support

       励志类文章       1.00      1.00      1.00         2
       科技类文章       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4



## TFIDF特征抽取

In [311]:
paper1 = "风月如流，光阴迈着一成不变的脚步，不疾不缓地走着，走过一个个春秋冬夏，走过一个个月阴日晴。蓦然回首，远逝的日子仿佛晃动的万花筒，不经意间摇晃出一个个无法模仿的图案，那一次次的聚散与悲欢都成绝版。于是我知道，青春是一列单程客车，不能返回也没有驿站。晨钟暮鼓，催促我扬起远航的风帆，心却如一枚留恋故枝的叶子，踟蹰在昨日的风中，依依难从我做起舍。也知道不该贻误行程，却又无法摆脱情感的羁绊，无法将曾经有过的一切尘封在记忆的城堡中，重生日锁紧，每天走过日落月升，仿佛伫立于清澈的河流中，那清清的河水就是我拥有而且仅仅拥有一次的生命啊，我却有太多的时候是立在水中，任凭上游的水流过我之后成为下游的水，任凭未来流经我的现在成为过去，我却因为过分沉迷于昨天而失去了今天和明天，于是一无所有。常常想象，当春华和秋实都成为历史之后，我披满头雪花坐在昨天的记忆中，咀嚼脸上每一道沟壑所深藏的故事，该是一种怎样的心境!本该是懂得很多便相信得很少，却依然一次次为“狼来了”的戏言所惑;本该是走过冬天更钟情于春天，却依然留恋“雪孩子”的童话。走过的路程似乎是一个圆，长途跋涉后又回到起点。于是很多时候是在做相同的事，就是寻找目标。曾经在幽幽暗暗中苦思冥想，终于破译出原来最简单的草枯草荣就是一种玄奥，那是生命的直观教具：花只能红一次，草只能绿一年。逝者如斯，如烟的往事都已随季风飘零，何必苦苦地追寻与回忆。青春的列车载客无数，没有人能预料沿途的遭遇，也没有人能重回自己上车的起点。既然已经别无选择，就让我在时间的钢轨上碾碎曾经有过的失意与如意，然后推开门窗，沐着新鲜的空气与阳光，迎接崭新的行程。因为我已经知道，青春是一列单程客车，不能返回也没有驿站。"
paper2 = "那些青春，已悄然老去;那些梦想，你可曾记得?青春，只一个很美的词，但又不知道它没在哪里。或许是因为年轻，所以享有青春。或许是因为年轻，才会对所有怀揣幻想。有些时候，曾幻想现实是多么美好，真想一下子融入现实生活中去感受一切。校园里的青涩年华，拥有美好的青春时光。我是幸福的，幸福到感受不到现实的残酷。而今，忽然回头，一些斑驳的记忆已支离破碎，再也拼凑不回。在我们的生命中，青春也悄然老去，离我们越拉越远，模糊了我们的实现，忘记了前路的方向。青春，是一个残酷的词。但又不明白它为何残酷。或许是因为成熟，所以知道残酷。或许是因为成熟，才会慢慢遗忘了青春。偶尔回头看看，却发现自己已偏离了轨道，和现实越走越近了，以至于丢失了那份纯真。梦想，似乎很重、很重。压得我们快要喘不过气。有时候甚至想丢开它，独自去一个无人知晓的地方。不知不觉中，梦想也随着我们长大了，越长大就越重。我们肩膀上的压力越大，脚步就不会变慢。这个季节，很快就会离我们而去，只是我们还停留在原地，依然守护着那些遥不可及的梦想。曾几何时，梦想拉近了我们与现实的距离，把我们一下子打回了原形。我们渐渐地，和现实越走越近。而在残酷的现实中，梦被扭曲了，变味儿了。无意之中，它渐渐偏离了我们。我们可以选择做一个自私的人，哪怕是一无所有。然后有一天突然发现，自己也变得很现实。于是只好重新拾起那些残缺的梦想。那些陈腐的过往，如今已随风即逝。雨过之后，我们看见的不是阳光，不是彩虹，而已灰色的天空，没有一丝色彩。繁花过后，青春散尽。那些要不可以的梦想，随着一阵微风而远行，不留一丝足迹。而在我们心底，仍然残留着一些破碎的痕迹。常说：我们有资本，因为我们还年轻。可是青春经不起肆意挥霍，也许有一天梦想会随着时间的流逝而不见。有一天，我们终将会慢慢老去。生活中，我们不应该是感伤的。忧伤和快乐事不过是一种形式。岁月带走了青春，别人看似美好的青春，但没有多少实际意义。奋斗，为时不晚，因为心中有梦。纵然年华不再，但是心中的斗志永不磨灭。它就像是一个火种，只要心底的那一丝温度不灭，它就会燃烧，越烧越旺。让我们接受过去，接受自己，带上梦想，悄悄地开始一段自己的旅程……"
paper3 = "四十年砥砺奋进，七十年长歌未央，一百年精神永存，时至今日，在五四爱国旗帜的引领下，中国人进步图存、科学救国。中国梦以一丝微光到今日的无限光明，离不开一代又一代人的接力传承，作为当代青年，我们承载着中华民族复兴之重任，科学兴国，吾辈担当。科学兴国，是历史的启迪。那时我们沉浸于“天朝上国”的美梦而不自知，殊不知，工业革命后科技发展迅速的西方文明已逐渐将我们甩在身后。当西方列强用坚船利炮轰开中国紧锁已久的大门，这片繁华千年的大地上留下了无尽的血泪与伤痕。近百年屈辱史，凝聚了中华民族无力的抗争与挣扎，终将沉睡的民族唤醒。鲁迅说：“我们将火药制成了烟花炮竹，可西方却将其制成火炮。”科学的停滞与落后，是造成我国国力落后于西方的根本原因，五四运动高举科学救国的旗帜，将中国唤醒。科学兴国，是时代的感召。我们关注今日之中国，经济飞速发展，已跃居世界前列。林达曾言：“刺破水面的锐利冰封，它也许只露出一角。”我们不仅仅要看到经济增长的高速度，更要看到速度之下，能源驱动型产业面临着资源枯竭、环境污染;劳动力密集型产业面临着人口红利衰退，增长乏力;高科技产业核心科技受制于发达国家的窘境。无论经济之花多么繁荣美丽，若没有科技作为支撑，中国经济持续发展，中国之梦的实现又何以为继?我们必须重新审视这个时代，这是一个科技飞速发展，日新月异的时代，科学技术作为第一生产力，已在无数实践中被证明，迈入知识经济时代，国之兴盛当以科学兴国、科学创新为先。科学兴国，是吾辈之担当。挪威戏剧家易卜生说过：“社会如同一条船，每个人都要做好掌舵的准备。”而我们要做的，则是要做好接力传承中华科学兴国之使命。我们可以看到，先辈们已为我们迈出了坚实一步，从五四科学救国的大旗，到“八六三”计划，科教兴国战略，再到神舟系列飞船升天，杂交水稻亩产再度突破。我看到，从邓稼先的隐姓埋名，到屠呦呦永不放弃，再到南仁东的一生坚守，无数前辈的努力，造就了今日蓬勃发展之中国。而我辈当承其志，将中国梦以高铁般速度持续推进。这是一个充满希望的国度，这是一个生机勃勃的时代，领袖在新年贺词中说：“我们都在努力奔跑，我们都是追梦人。”愿当代青年同胞们用科学力量武装自己，接力传承中华复兴之梦!"
paper4 = "开学第一课，一听到这个名字，每一个小学生都会记忆犹新，而他也整整陪伴了我5年。今年的主题是五星红旗，我为你自豪！说到五星红旗，有的人就会想到战争时期战士们的英勇牺牲；有的人就会想到党中央的正确领导；还有的人会想到中国刚成立百废待兴的时刻；而我想到的是我们国家日新月异的科技发展。这70年里，我国的科学技术从落后到了前三，从一无所有到世界领先，今年上半年，在第四次工业革命中，我们实现了弯道超车。中国的科技正在飞速发展，新四大发明正在影响着世界。在开学第一课里，那些大哥哥大姐姐手中的救援机器人成功实现了3分钟的迷你救援。看到他们将拼好的小机器人通过ipad操控就可以操控自如。尽管路上有很多障碍物，但是这些机器人靠这搭桥车通过断桥，靠推作文https://Www.ZuoWEn8.Com/土车把石头推到一边，靠机器爪子把一大块的墙移开救出人，然后送进车里飞快的去医院。我们都知道，当灾难降临后的72小时是救援的最好时机也就是3天，由于是迷你实验的成功也可以预测出我们在真实情况下也可以在3天之内救出被困者。我们不光有先进的救援设备，我们还有探索太空的神器——玉兔，当那个小家伙出现在舞台上的时候，我的眼睛都看直了。我也很想亲自到现场去看看，亲自操作一下小机器人，亲自和小玉兔进行互动……而更值得我学习的就是这些伟大的科技的背后，那个伟大的中国梦。所以作为巩华追梦少年的我要从新学期开始就努力严格要求自己，写一手好字，练就一副好口才，真正把爱国之志变成报国之行。今天为振兴中华而勤奋学习，明天为创造祖国辉煌未来贡献自己的力量！"
papers=[paper1,paper2,paper3,paper4]
#特征抽取
features=[]
for paper in papers:
    features.append(" ".join(list(filter(lambda x: x not in biaodian, jieba.cut(paper)))))
tfidf = TfidfVectorizer()
features = tfidf.fit_transform(features)
print(features.toarray())
print(tfidf.get_feature_names())

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.0380317  0.0380317  0.0380317 ]
 [0.06018309 0.06018309 0.06018309 ... 0.         0.         0.        ]]
['70', '72', 'com', 'https', 'ipad', 'www', 'zuowen8', '一下', '一下子', '一下子打', '一丝', '一个', '一个个', '一些', '一代', '一代人', '一切', '一列', '一副', '一大块', '一天', '一年', '一成不变', '一手', '一无所有', '一条', '一枚', '一次', '一次次', '一步', '一段', '一生', '一百年', '一种', '一角', '一边', '一道', '一阵', '七十年', '万花筒', '上半年', '上游', '上车', '下游', '不仅仅', '不会', '不光', '不再', '不到', '不回', '不是', '不留', '不疾不缓', '不知不觉', '不经意', '不能', '不见', '不该', '不过', '世界', '世界领先', '丢失', '丢开', '严格要求', '中华', '中华民族', '中国', '中说', '为何', '为先', '为时不晚', '主题', '之下', '之内', '之后', '之志', '之梦', '之花', '乏力', '也许', '于是', '互动', '五四', '五四运动', '五星红旗', '产业', '亩产', '享有', '亲自', '人会', '人口', '人能', '仅仅', '今天', '今年', '今日', '仍然', '从我做起', '他们', '以至于', '任凭', '仿佛', '伟大', '传承', '伤痕', '伫立', '似乎', '但是', '何以', '何必'

In [324]:
# tfidf案例

In [347]:
from sklearn.datasets import fetch_20newsgroups
newsgroup = fetch_20newsgroups(data_home="./data")
dir(newsgroup)

['DESCR', 'data', 'filenames', 'target', 'target_names']

In [348]:
# 特征抽取
tfidf_vect = TfidfVectorizer()
features = tfidf_vect.fit_transform(newsgroup.data)

In [349]:
# train_test_split
x_train,x_test,y_train,y_test = train_test_split(features, newsgroup.target, train_size=0.90)

In [350]:
# 训练
multi = MultinomialNB()
multi.fit(x_train,y_train)
multi.score(x_test,y_test)

0.8604240282685512